In [65]:
import agents
import game
import mechanisms

import numpy as np

In [66]:
import importlib
importlib.reload(agents)
importlib.reload(game)
importlib.reload(mechanisms)

<module 'mechanisms' from 'C:\\Users\\billz\\PycharmProjects\\CS_281\\Finding_Friends\\mechanisms.py'>

In [86]:
NUM_PLAYERS = 5
BASE_LVL = 0
LVL_CAP = 100
WIN_PROB = 1.0

skill_levels = np.random.randint(1, 5, size=NUM_PLAYERS)
print(skill_levels)

ff_agents = [agents.Basic_Agent(i, BASE_LVL) for i in range(NUM_PLAYERS - 1)]
# ff_agents.append(agents.Lowest_Level_Agent(NUM_PLAYERS - 3, BASE_LVL))
# ff_agents.append(agents.Lowest_Level_Agent(NUM_PLAYERS - 2, BASE_LVL))
ff_agents.append(agents.Lowest_Level_Agent(NUM_PLAYERS - 1, BASE_LVL))

ff_mech = mechanisms.Baseline_Mechanism(num_players=NUM_PLAYERS, p=WIN_PROB)
# ff_mech = mechanisms.Skill_Mechanism(num_players=NUM_PLAYERS, skill_levels=skill_levels)

ff_game = game.Game(players=ff_agents, mechanism=ff_mech, cap=LVL_CAP, logging_level='DEBUG')

[3 3 1 3 3]


In [87]:
NUM_TRIALS = 200
results = []

for i in range(NUM_TRIALS):
    ff_game.play()
    results.append(ff_game.levels)
    ff_game.reset()

results

[[100, 93, 93, 94, 94],
 [99, 100, 99, 99, 95],
 [99, 100, 97, 99, 99],
 [90, 91, 89, 90, 100],
 [100, 97, 99, 96, 94],
 [90, 89, 89, 88, 100],
 [89, 92, 88, 89, 100],
 [100, 96, 95, 96, 87],
 [97, 99, 99, 100, 93],
 [95, 100, 97, 95, 97],
 [97, 96, 96, 100, 83],
 [92, 94, 91, 93, 100],
 [91, 92, 94, 97, 100],
 [97, 97, 100, 100, 92],
 [96, 94, 91, 91, 100],
 [95, 93, 94, 100, 94],
 [100, 98, 98, 99, 97],
 [97, 99, 98, 100, 100],
 [98, 97, 100, 97, 84],
 [100, 99, 98, 98, 93],
 [95, 89, 88, 88, 100],
 [90, 88, 89, 89, 100],
 [100, 96, 97, 96, 93],
 [92, 94, 92, 92, 100],
 [97, 98, 98, 99, 100],
 [100, 96, 98, 97, 97],
 [88, 86, 85, 89, 100],
 [97, 97, 100, 95, 99],
 [94, 94, 96, 94, 100],
 [97, 98, 96, 97, 100],
 [94, 94, 94, 98, 100],
 [96, 94, 93, 95, 100],
 [94, 94, 100, 93, 97],
 [94, 94, 93, 93, 100],
 [91, 100, 91, 91, 99],
 [100, 100, 96, 95, 91],
 [100, 93, 92, 92, 87],
 [97, 100, 99, 97, 93],
 [78, 79, 78, 77, 100],
 [100, 98, 96, 96, 90],
 [100, 97, 97, 95, 93],
 [93, 94, 93,

In [88]:
np.sum([results[i][-1] == 100 for i in range(len(results))]) / NUM_TRIALS

0.455